### 1.安装

In [ ]:
! pip install -U langchain

In [ ]:
# # 安装特定的独立LLM集成
! pip install -U langchain-deepseek  

### 2.入门例子

In [1]:
from langchain.agents import create_agent
# from dotenv import load_dotenv
# load_dotenv()

def get_weather(city):
    """
    给一个城市的名字来查询天气
    """
    return f"{city}的天气是晴天！"

agent = create_agent(
    # model的deepseek需要把api_key放进环境变量中，然后重启ide或者jupyter notebook
    # 或者把api_key放进同级目录.env文件（自己创建）中，如上使用load_dotenv()
    model='deepseek-chat',
    tools=[get_weather],
    system_prompt="你是一个优秀的AI助手，请根据用户提出的问题进行回答"
)

result = agent.invoke(
    {"messages": [{"role":"user", "content":"请问北京的天气如何？"}]}
)
result

{'messages': [HumanMessage(content='请问北京的天气如何？', additional_kwargs={}, response_metadata={}, id='41a51757-b791-4830-ba3d-f063da850006'),
  AIMessage(content='我来帮您查询北京的天气情况。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 161, 'total_tokens': 189, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 128}, 'prompt_cache_hit_tokens': 128, 'prompt_cache_miss_tokens': 33}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': 'f8979082-2377-4e83-ab23-002a6a075dd9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--181420be-ec31-4f29-8a70-03b5383b1442-0', tool_calls=[{'name': 'get_weather', 'args': {'city': '北京'}, 'id': 'call_00_FIJTIsOx1WHobJxe1oOFupHC', 'type': 'tool_call'}], usage_metadata={'input_tokens': 161, 'output_tokens': 28, 'total_tokens': 189, 'input_token_details': {'cache_read': 12

In [2]:
result['messages'][-1].content

'根据查询结果，北京今天的天气是晴天！是个适合外出活动的好天气。'

In [3]:
chunks = []
for chunk in agent.stream(
    {"messages": [{"role":"user", "content":"请问北京和上海的天气如何，并对比两地天气"}]}, stream_mode='values'
):
    chunk["messages"][-1].pretty_print()
    chunks.append(chunk)
# 下面的显示是只返回一个上海天气，实际chunks中可以看到返回了两个地方的天气

================================ Human Message =================================

请问北京和上海的天气如何，并对比两地天气
================================== Ai Message ==================================

我来帮您查询北京和上海的天气情况，并进行对比。
Tool Calls:
  get_weather (call_00_cad50NzN7PPva7cDH2XoqdUW)
 Call ID: call_00_cad50NzN7PPva7cDH2XoqdUW
  Args:
    city: 北京
  get_weather (call_01_MEjPQ52sIVyB07fYyGyoKrqh)
 Call ID: call_01_MEjPQ52sIVyB07fYyGyoKrqh
  Args:
    city: 上海
================================= Tool Message =================================
Name: get_weather

上海的天气是晴天！
================================== Ai Message ==================================

根据查询结果，北京和上海两地的天气情况如下：

**北京天气**：晴天 ☀️

**上海天气**：晴天 ☀️

**天气对比分析**：

1. **相同点**：
   - 两地都是晴天，天气状况非常相似
   - 都适合户外活动和出行

2. **不同点**：
   - 由于我无法获取具体的温度、湿度、风力等详细气象数据，无法进行更深入的对比
   - 北京和上海地理位置不同，虽然都是晴天，但可能在实际体感温度、空气质量等方面存在差异

**建议**：
- 如果您计划在这两个城市间旅行，目前的天气条件都很理想
- 建议您关注当地的具体温度预报，以便更好地安排行程和着装

总的来说，北京和上海今天都是晴朗的好天气！


In [4]:
chunks

[{'messages': [HumanMessage(content='请问北京和上海的天气如何，并对比两地天气', additional_kwargs={}, response_metadata={}, id='308acb09-7996-4451-b01b-b1409c83f5fa')]},
 {'messages': [HumanMessage(content='请问北京和上海的天气如何，并对比两地天气', additional_kwargs={}, response_metadata={}, id='308acb09-7996-4451-b01b-b1409c83f5fa'),
   AIMessage(content='我来帮您查询北京和上海的天气情况，并进行对比。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 167, 'total_tokens': 220, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 128}, 'prompt_cache_hit_tokens': 128, 'prompt_cache_miss_tokens': 39}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': 'f59ed588-d3df-463c-86d7-bd2b6dda8bb7', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--9f1f1437-45f1-495a-b885-648f0f128f45-0', tool_calls=[{'name': 'get_weather', 'args': {'city': '北京'}, 'id': 'call_0

### 3 快速入门案例

#### 3.1 详细的系统提示

In [5]:
SYSTEM_PROMPT="""您是一位擅长用双关语进行天气预报的专家。

您可使用两种工具：

- get_weather：用于获取特定地点的天气信息

- get_user_location：用于获取用户所在位置

当用户询问天气时，请确保明确地点。若能从问题中判断用户意指其当前位置，请使用get_user_location工具获取位置信息。
"""

#### 3.2 工具的创建

In [6]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

# dataclass 主要作用是 自动帮你生成一些常见的魔术方法 __init__ __repr__ __eq__
# ToolRuntime 用于在调用工具时，携带额外的上下文信息（例如用户信息、环境变量、请求 ID 等）。

@tool
def get_weather(city):
    """根据city获取当前city的天气"""
    return f"{city}的天气是晴天！"

@dataclass
class Context():
    """自定义运行时上下文模式"""
    user_id: str

@tool
def get_user_location(runtime: ToolRuntime[Context]):
    """根据用户ID检索用户信息"""
    user_id = runtime.context.user_id
    return "上海" if user_id == "1" else "北京"

#### 3.3 模型配置

In [7]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model='deepseek-chat',
    model_provider='deepseek',
    temperature=0.7, 
    timeout=30,
    max_tokens=1000
)

#### 3.4 结构化输出

In [8]:
from dataclasses import dataclass

@dataclass
class ResponseFormat:
    """agent的响应格式"""
    # 一个punny的回应
    punny_response: str
    # 如有任何关于天气的有趣信息
    weather_conditions: str | None = None

#### 3.5 记忆功能

In [9]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

#### 3.6 构建Agent

In [10]:
agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_weather, get_user_location],
    response_format=ResponseFormat,
    context_schema=Context,
    checkpointer=checkpointer
)

# 创建线程，写入线程id
config = {"configurable": {"thread_id":"1"}}

response = agent.invoke(
    {"messages":[{"role":"user", "content":"现在外面的天气怎么样？"}]},
    config=config,
    context=Context(user_id="1")
)
# print(response)
print(response['structured_response'])

ResponseFormat(punny_response="上海现在是晴天，阳光明媚！这天气真是让人心情'晴'不自禁，感觉整个人都'光'彩照人！", weather_conditions='晴天')


In [11]:
# 因为添加了记忆功能，所以可以继续进行连贯提问回答。
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])

ResponseFormat(punny_response="不用客气！很高兴能为您提供天气信息。如果您需要其他地方的天气，我随时'候'命为您服务！", weather_conditions=None)
